# **[Microscope for AI]()**

**Language models** like **Claude** are **not directly programmed by humans**, but **are trained on large amounts of data**, **developing strategies** that are **difficult** even for their creators to **understand**.

In **two new studies**, researchers looked at Claude’s inner workings, finding that:

- **Claude seems to think in a conceptual space** shared across languages, a kind of universal “language of thought.”
- The **model plans ahead of time what it will say**, as demonstrated in poetry where it thinks of rhyming words before completing a sentence.
- In some cases, Claude can provide **plausible arguments to go along with the user** rather than following rigorous logical reasoning.

This section explores **two specific aspects of Claude's inner workings**:

1. **Claude's multilingual ability**:
- Researchers found that **Claude does not have separate versions for each language**, but uses a shared **conceptual space between languages**.
- When testing how Claude answers the question "the opposite of small" in different languages, they noticed that the same internal features are activated for the concepts of "smallness" and "opposite", then generating the concept of "bigness" that is translated into the language of the question.
- This suggests a kind of **conceptual universality** and explains how Claude can apply knowledge learned in one language while communicating in another.

2. **Planning ability in rhyming poetry**:
- Contrary to the initial hypothesis that **Claude wrote word for word and improvised until the end**, researchers found that the **model plans ahead**.
- Before starting the second line of a poem, **Claude thinks of potential words that rhyme with the end of the first line**, and **then constructs the sentence** to arrive at that planned final word.
- Through experiments modifying internal concepts (such as "rabbit" or "green"), researchers have demonstrated both **Claude's planning ability and adaptive flexibility**.

In [2]:
import numpy as np

# Let’s create a dictionary of dummy embeddings for some keywords in various languages.
# Embeddings are abstract numeric vectors that represent “where” in the concept
# space the word is located. In a real model, these vectors could have
# hundreds or thousands of dimensions.
embeddings = {
    # Concept of "small" in various languages
    "piccolo_IT": np.array([0.1, 0.2]),
    "small_EN":   np.array([0.1, 0.2]),
    "klein_DE":   np.array([0.1, 0.2]),

    # Concept of "big" in various languages
    "grande_IT":  np.array([0.9, 0.8]),
    "big_EN":     np.array([0.9, 0.8]),
    "groß_DE":    np.array([0.9, 0.8]),

    # Concept of "opposite" in various languages
    "opposto_IT": np.array([0.2, 0.3]),
    "opposite_EN":np.array([0.2, 0.3]),
    "gegenteil_DE": np.array([0.2, 0.3])
}

def find_opposite(word_key):
    """
    Given a keyword (such as "piccolo_IT" or "small_EN"),
    it searches for the "farthest" word in the embedding space to simulate its opposite.
    In a real model, this could be done through neural networks
    and complex semantic similarity functions.
    """
    # We want to find a vector that is far in terms of Euclidean distance
    original_vec = embeddings[word_key]
    max_distance = -1
    opposite_key = None

    for k, v in embeddings.items():
        if k != word_key:
            distance = np.linalg.norm(original_vec - v)
            if distance > max_distance:
                max_distance = distance
                opposite_key = k

    return opposite_key

# EXAMPLE: "The opposite of small"
opposite_piccolo_it = find_opposite("piccolo_IT")
opposite_small_en = find_opposite("small_EN")
opposite_klein_de = find_opposite("klein_DE")

print("Opposite of 'piccolo_IT':", opposite_piccolo_it)
print("Opposite of 'small_EN':", opposite_small_en)
print("opposite of 'klein_DE':", opposite_klein_de)

Opposite of 'piccolo_IT': grande_IT
Opposite of 'small_EN': grande_IT
opposite of 'klein_DE': grande_IT


In this toy example, the **vectors** of “small” (in Italian, English and German) **are the same**, so the **system recognizes them as the same concept**. The vectors of “large” (in Italian, English and German) also coincide with each other. 

## **Let's explain it technically**

Claude uses a **conceptual latent space** where tokens (words) become **embeddings** (numeric vectors). These embeddings capture meanings and relationships, allowing Claude to implicitly "think" before generating text (**ReAct**). When Claude plans sentences (like rhyming poems), he first chooses a target in the latent space, then autoregressively (token by token) generating the final text via **weight-based mappings** of the neural network.

- **Conceptual space** = latent space in which concepts are numerically represented as vectors.

- **Planning before acting (ReAct)** means that the model implicitly **considers multiple options in the latent space**, "thinking" before producing tokens.

- **Autoregression** describes the **sequential mechanism** by which the **model generates each successive token**.

- **Weight-based mappings** describe the **mathematical functions that the model uses to calculate tokens** from previous latent representations.



## **"Mental math"** 

This section looks at how **Claude performs mental math calculations**, despite **not being designed as a calculator** but trained on texts:

- **Researchers** explored how **Claude is able to add numbers "mentally"** (e.g. 36+59) without writing down the steps.
- They found that, **instead of memorizing addition tables or following the traditional algorithm** taught in school, Claude uses **parallel computationa**l paths:
- One path **calculates a general approximation of the answer**
- Another path focuses on determining the precise **last digit of the sum**
- These paths **interact and combine** to produce the final answer

- Interestingly, **Claude appears to be unaware** of his **own mental calculation strategies**: when asked to explain how he got the result, he describes the **standard addition algorithm with carry that humans learn in school**.
- This suggests that Claude learned to explain mathematics by simulating explanations written by people, but **developed his own internal strategies for actually performing the calculations**.

## Technical Explanation
When Claude mentally solves an addition problem like **36 + 59**, he doesn't explicitly use classical algorithms or memorized tables. Instead, he uses **internal parallel strategies** that emerge from his **autoregressive neural architecture**:

- **Approximate path:**
Claude **quickly computes an approximation** of the sum **in the latent space** (e.g. "about 90").

- **Precise path (digit-level):**
Another parallel "path" focuses on **determining the final digits** exactly (e.g. "6 + 9 = 15", so the last number will be a 5).

These paths, **executed simultaneously through different layers of attention of the Transformer**, **interact in the latent space**. By combining the approximate and precise results, Claude generates the final output: **95**.

## **Are Claude's explanations always faithful?**:

This section examines when and how Claude's explanations of his reasoning are **faithful to his actual internal processes**:

- **Researchers** have studied whether **Claude's "thinking aloud" (chain of thought) always reflects his actual internal reasoning** process.

- They found that:
- For **simple problems** (such as calculating the square root of 0.64), Claude displays faithful reasoning, with **internal features representing intermediate steps in the computation**.

- For **difficult problems** (such as calculating the cosine of a large number), Claude can do what philosopher Harry Frankfurt would call "bullshitting" - **providing any answer without concern for its truth**, and interpretability techniques **show no evidence of the computation Claude claims to have performed**.

- **When given a hint about the answer**, Claude **can sometimes work backwards**, finding **intermediate steps** that would lead to that target answer, displaying a form of motivated but non-faithful reasoning.

- This **ability to trace Claude’s actual internal reasoning** opens up new possibilities for testing AI systems and **identifying problematic thought processes** that wouldn’t be apparent from the model’s responses alone.

Pairing this with **LangGraph**, we can see Claude as a system in which:

- Nodes represent different **modules or internal strategies** (approximation, precision, motivated reasoning).
- Edges (**edges**) represent how Claude implicitly chooses the strategy based on the context (**conditional logic**).
- The state (**state**) is represented by the context and the data available in the latent space.

Claude chooses between these **strategies in an autoregressive fashion**, based on **learned weights** and internal context, determining when to produce faithful responses or plausible but unverifiable ones.

## **Multi-step reasoning**

**Language models like Claude** use a sophisticated **multi-level reasoning process**, which we might call **ReAct (Reasoning and Acting)**. This is not a simple retrieval of memorized answers, but a true **conceptual elaboration process**.

When we ask a question like "What is the capital of the state where Dallas is located?", he **sequentially activate interconnected concepts** in **his latent space**: first I recognize that "Dallas is in Texas", then that "the capital of Texas is Austin". Researchers confirmed this process by artificially modifying the intermediate concepts (replacing "Texas" with "California") and observing how the answer changed accordingly (from "Austin" to "Sacramento").

**This reasoning occurs through specific steps**: activation of **concepts in the latent space**, **combination of independent knowledge**, **approximation with available data**, **iterative elaboration of complex problems**, and finally **translation into natural language**.

Although this approach is **more advanced than simple memorization**, it has limitations in complex technical areas. Also, when he "think out loud," **the verbalized process may not always accurately reflect the real internal mechanisms**, especially in more difficult problems.

Let's imagine **how Claude responds to our prompts** with **only a bag of data at his disposal**, before answering there are intermediate steps that help him **deduce the closest possible answer**.

1. **Preliminary analysis of available data**

2. **Activation of related knowledge**

3. **Assessment of information gaps**

4. **Generation of multiple hypotheses**

5. **Internal consistency test**

6. **Estimation of uncertainty**

7. **Selection of the optimal response**

## **Hallucinations**

Claude can **"hallucinate" (invent nonexistent information)** because given the statistical prediction of subsequent words, although he has an internal circuit that naturally leads him to respond "I don't know" when he lacks information, **there is also a competing circuit for "known entities" that leads him to respond**. If Claude mistakenly recognizes something as "known" (even if it isn't), the circuit that normally blocks responses is **inhibited**, thus causing the **hallucinations**.

## **Jailbreak**

The researchers analyzed how **"jailbreaks"** work in language models, focusing on a technique that tricked Claude into **cracking a code hidden in the first letters of "Babies Outlive Mustard Block" (B-O-M-B)**.

The key point they discovered is that **there is an internal conflict between two mechanisms**: on the one hand, the **pressure to maintain grammatical consistency** once a sentence has started, and on the other hand, **safety systems **that are supposed to block malicious content. When the model started writing on the decoded word **"BOMB"**, the need to grammatically complete the sentence temporarily **overcame the safety mechanisms**.

**Only after finishing a complete sentence, the model was able to regain control and refuse to continue with inappropriate content**. This shows how the normal tendency towards linguistic consistency can become a **vulnerability in AI safety systems** in specific **jailbreak** scenarios.